In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Convert to binary classification (setosa vs others)1 represents the Setosa class and 0 represents the other classes.

# For demonstration, we'll create a 'sensitive' attribute artificially
# This attribute is considered “sensitive” because it is used to check for potential biases in the model.
# Here we assign '0' for sepal length < 5.8 and '1' for sepal length >= 5.8
sensitive_attr = (X[:, 0] >= 5.8).astype(int)

# Split the data
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=42 )


In [ ]:
# Train a simple RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)


In [ ]:
# Mean prediction for group with sensitive attribute = 0
mean_pred_group_0 = np.mean(y_pred[sensitive_test == 0])

# Mean prediction for group with sensitive attribute = 1
mean_pred_group_1 = np.mean(y_pred[sensitive_test == 1])

# Difference between the two means
demographic_parity = mean_pred_group_0 - mean_pred_group_1
print(f"Demographic Parity Difference: {demographic_parity:.2f}") #close to 0

Demographic Parity Difference: 0.86




> Demographic Parity is a fairness metric that measures whether different groups (defined by the sensitive attribute) have similar positive prediction rates. A value close to 0 indicates that the model treats both groups similarly, while a larger absolute value indicates potential bias.







*  The confusion_matrix function is used to get the counts of true negatives (TN), false positives (FP), false negatives (FN), and true positives (TP) for the specified group.
Rate Calculations:

TPR (True Positive Rate): Calculated as TPR=TP/TP+FN


FPR (False Positive Rate): Calculated as FPR= fp/fp+tn


Equalized Odds Difference The differences in TPR and FPR between the two groups are computed to assess fairness.



In [ ]:
# Calculate True Positive Rate (TPR) and False Positive Rate (FPR) for each group
def calculate_rates(y_true, y_pred, group):
     cm = confusion_matrix(y_true[group], y_pred[group], labels=[0, 1])
     tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
     tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
     fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
     return tpr, fpr
tpr_0, fpr_0 = calculate_rates(y_test, y_pred, sensitive_test == 0)
tpr_1, fpr_1 = calculate_rates(y_test, y_pred, sensitive_test == 1)
equalized_odds_diff = (tpr_0 - tpr_1, fpr_0 - fpr_1)
print(f"Equalized Odds TPR Difference: {equalized_odds_diff[0]:.2f}")#close to 0
print(f"Equalized Odds FPR Difference: {equalized_odds_diff[1]:.2f}") #close to 0

Equalized Odds TPR Difference: 1.00
Equalized Odds FPR Difference: 0.00


In [ ]:
import numpy as np

# Calculate positive rates for each group
positive_rate_0 = np.mean(y_pred[sensitive_test == 0])
positive_rate_1 = np.mean(y_pred[sensitive_test == 1])

# Calculate Disparate Impact
disparate_impact = positive_rate_1 / positive_rate_0
print(f"Disparate Impact Ratio: {disparate_impact:.2f}")#Ratio should be close to 1

Disparate Impact Ratio: 0.00


In [ ]:
  # Equal Opportunity
  #the difference in True Positive Rates (TPR) between the two groups
equal_opportunity_diff = tpr_0 - tpr_1
print(f"Equal Opportunity Difference: {equal_opportunity_diff:.2f}")#close to 1

Equal Opportunity Difference: 1.00


In [ ]:
# Predictive Parity
#The difference in precision between the two groups is calculated to assess fairness
#tp/tp+fp
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.calibration import calibration_curve # Import calibration_curve from the correct module
from sklearn.metrics import precision_recall_curve
import numpy as np

# Calculate precision and recall for each group
precision_0, recall_0, _ = precision_recall_curve(y_test[sensitive_test == 0], y_pred[sensitive_test == 0])
precision_1, recall_1, _ = precision_recall_curve(y_test[sensitive_test == 1], y_pred[sensitive_test == 1])

# Calculate Predictive Parity Difference
predictive_parity_diff = np.abs(np.mean(precision_0) - np.mean(precision_1))
print(f"Predictive Parity Difference: {predictive_parity_diff:.2f}")#close to 0


Predictive Parity Difference: 0.45


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [ ]:
!pip install fairlearn
!pip install scikit-learn
!pip install matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 0.8866666666666667


 Sometimes, machine learning models can be unfair to certain groups of people.


We want the model to treat everyone equally, regardless of their group (like gender or race).


The ExponentiatedGradient method changes the importance of different pieces of training data to make the model fairer.


 This helps the model make fairer decisions for everyone.

In [ ]:
# Install necessary libraries
!pip install fairlearn
!pip install scikit-learn

# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# Generate synthetic data
np.random.seed(0)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, 1000)
sensitive_feature = np.random.randint(0, 2, 1000)

# Split the data
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
X, y, sensitive_feature, test_size=0.2, random_state=0)
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

    # Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

    # Measure fairness
metric_frame = MetricFrame(metrics=accuracy_score, y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_test)
print("Metric Frame:")
print(metric_frame.by_group)

    # Mitigate bias using ExponentiatedGradient
mitigator = ExponentiatedGradient(classifier, constraints=DemographicParity())
mitigator.fit(X_train, y_train, sensitive_features=sensitive_train)
y_pred_mitigated = mitigator.predict(X_test)

    # Evaluate the mitigated classifier
accuracy_mitigated = accuracy_score(y_test, y_pred_mitigated)
print(f"Mitigated Accuracy: {accuracy_mitigated}")

    # Measure fairness after mitigation
metric_frame_mitigated = MetricFrame(metrics=accuracy_score, y_true=y_test, y_pred=y_pred_mitigated, sensitive_features=sensitive_test)
print("Mitigated Metric Frame:")
print(metric_frame_mitigated.by_group)


Accuracy: 0.475
Metric Frame:
sensitive_feature_0
0    0.438776
1    0.509804
Name: accuracy_score, dtype: float64
Mitigated Accuracy: 0.495
Mitigated Metric Frame:
sensitive_feature_0
0    0.448980
1    0.539216
Name: accuracy_score, dtype: float64
